In [1]:
import tensorflow as tf 
import keras_cv
from tensorflow import keras 
from keras import layers, Input

  
import numpy as np 
import matplotlib.pyplot as plt 
  


# Load in the data 
cifar100 = tf.keras.datasets.cifar100 

# Distribute it to train and test set 
(x_train, y_train), (x_test, y_test) = cifar100.load_data() 
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape) 


2024-04-17 12:36:10.402231: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 12:36:10.402359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 12:36:10.553206: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [ ]:
model = tf.keras.models.Sequential([ 
layers.Conv2D(16, (3, 3), activation='gelu',input_shape=(32, 32, 3), padding='same'), 
layers.MaxPooling2D(2, 2), 
layers.Conv2D(32, (3, 3),activation='gelu', padding='same'), 
layers.MaxPooling2D(2, 2), 
layers.Conv2D(64, (3, 3), activation='gelu', padding='same'), 
layers.MaxPooling2D(2, 2), 
layers.Conv2D(128, (3, 3), activation='gelu', padding='same'), 


layers.Flatten(), 
layers.Dense(256, activation='gelu'), 
layers.BatchNormalization(), 
layers.Dense(256, activation='gelu'), 
layers.Dropout(0.3), 
layers.BatchNormalization(), 
layers.Dense(100, activation='gelu') 
]) 

model.summary()

In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, AveragePooling2D
from tensorflow.keras.layers import Add, DepthwiseConv2D, Resizing
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau

def inverted_residual_block(x, lf, expand=64, squeeze=16):
  m = Conv2D(expand, (1,1), activation='gelu', padding= "same")(x)
  m = DepthwiseConv2D(lf, activation='gelu', padding= "same")(m)
  m = Conv2D(squeeze, (1,1), activation='gelu', padding= "same")(m)
  return Add()([m, x])

input_img = Input(shape=(32,32,3))

#shortcut 1
x_short1 = Conv2D(32, kernel_size=(3,3), padding= "same", activation='gelu')(input_img)
x_short1 = Resizing(8, 8, interpolation="bilinear", crop_to_aspect_ratio=True)(x_short1)

x = layers.Conv2D(16, (3, 3), activation='gelu',input_shape=(32, 32, 3), padding='same')(input_img)
x = inverted_residual_block(x, (3,3), expand=64, squeeze=16) #mobilenet layer
x = layers.MaxPooling2D(2, 2)(x)
x = layers.Conv2D(32, (3, 3),activation='gelu', padding='same')(x) 
x = inverted_residual_block(x, (3,3), expand=64, squeeze=32) #mobilenet layer
x = layers.MaxPooling2D(2, 2)(x)

x = Add()([x, x_short1])

x = layers.Conv2D(64, (3, 3), activation='gelu', padding='same')(x)
x = inverted_residual_block(x, (3,3), expand=128, squeeze=64) #mobilenet layer
x = layers.MaxPooling2D(2, 2)(x)

#shortcut 2
x_short2 = Conv2D(128, kernel_size=(3,3), padding= "same", activation='gelu')(input_img)
x_short2 = Resizing(4, 4, interpolation="bilinear", crop_to_aspect_ratio=True)(x_short2)

x = layers.Conv2D(128, (3, 3), activation='gelu', padding='same')(x) 
x = inverted_residual_block(x, (3,3), expand=192, squeeze=128) #mobilenet layer

x = Add()([x, x_short2])

x = layers.Conv2D(128, (3, 3), activation='gelu', padding='same')(x) 
x = inverted_residual_block(x, (3,3), expand=192, squeeze=128) #mobilenet layer

x = layers.Flatten()(x) 
x = layers.Dense(256, activation='gelu')(x) 
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation='gelu')(x) 
x = layers.Dropout(0.3)(x)
x = layers.BatchNormalization()(x)
output = layers.Dense(100, activation='gelu')(x)


model = Model(inputs=input_img, outputs=[output])
model.summary()

In [2]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# number of classes
nb_classes = len(np.unique(y_train))

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, nb_classes)
y_test = to_categorical(y_test, nb_classes)

# Normalize data
#med_train = np.mean(x_train)
#std_train  = np.std(x_train)
#med_test = np.mean(x_test)
#std_test  = np.std(x_test)

#x_train = (x_train - med_train) / std_train
#x_test = (x_test - med_test) / std_test

#x_train =(x_train)/255.0
x_test = (x_test)/255


train_datagen = ImageDataGenerator(
        #rotation_range= 40,
        rescale=1./255,
        vertical_flip=True,
        horizontal_flip=True)


it = train_datagen.flow(x_train, y_train, shuffle=False)
print(x_train.shape)
print(y_train.shape)


(50000, 32, 32, 3)
(50000, 100)


In [3]:
print (np.max(x_train), np.min(x_train))

255 0


In [4]:
from tensorflow.keras.models import Model
import tensorflow_hub as hub

model_eff = keras_cv.models.EfficientNetV2Backbone.from_preset(
   "efficientnetv2_s_imagenet"#"efficientnetv2_b2_imagenet",
)

# model_eff.trainable = False

model_convnext = keras.applications.ConvNeXtTiny(
    model_name="convnext_tiny",
    include_top=None,
    #include_preprocessing=True,
    weights="imagenet",
    input_tensor=None,
    #input_shape=(32,32,3),
    pooling=None,
    #classes=100,
    #classifier_activation="softmax",
)

input_img = Input(shape=(32,32,3))

x = model_eff(input_img)
#x = model_convnext(input_img)
#x = layers.GlobalAveragePooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='gelu')(x) 
x = layers.Dropout(0.3)(x)
x = layers.BatchNormalization()(x)
output = layers.Dense(100, activation='softmax')(x)

model = Model(inputs=input_img, outputs=[output])

model.summary()

Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_s_imagenet/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_s_imagenet/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/efficientnetv2/keras/efficientnetv2_s_imagenet/2' to your Kaggle notebook...


111650432/111650432 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficient_net_v2s_backbone      │ (None, 1, 1, 1280)     │    20,331,360 │
│ (EfficientNetV2Backbone)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        51,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,040,580 (80.26 MB)

 Trainable params: 20,885,684 (79.67 MB)

 Non-trainable params: 154,896 (605.06 KB)

In [5]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=1e-7)

model.compile( 
loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), 
#optimizer= keras.optimizers.Adam(learning_rate=1e-4),
optimizer= keras.optimizers.Lion(learning_rate=1e-4),
metrics=['accuracy']
) 


hist = model.fit(it,
                 epochs=30, 
                 batch_size=128, 
                 verbose=1,
                 callbacks=[reduce_lr],
                 validation_data=(x_test, y_test)) 

Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1713357626.584878      85 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 446s 155ms/step - accuracy: 0.1054 - loss: 4.2367 - val_accuracy: 0.3692 - val_loss: 2.4111 - learning_rate: 1.0000e-04
Epoch 2/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 85s 54ms/step - accuracy: 0.3509 - loss: 2.5004 - val_accuracy: 0.4790 - val_loss: 1.8818 - learning_rate: 1.0000e-04
Epoch 3/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 83s 53ms/step - accuracy: 0.4608 - loss: 1.9795 - val_accuracy: 0.5341 - val_loss: 1.6809 - learning_rate: 1.0000e-04
Epoch 4/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 84s 54ms/step - accuracy: 0.5320 - loss: 1.7044 - val_accuracy: 0.5665 - val_loss: 1.5838 - learning_rate: 1.0000e-04
Epoch 5/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 84s 53ms/step - accuracy: 0.5776 - loss: 1.4915 - val_accuracy: 0.5861 - val_loss: 1.5579 - learning_rate: 1.0000e-04
Epoch 6/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 85s 55ms/step - accuracy: 0.6199 - loss: 1.3244 - val_accuracy: 0.5939 - val_loss: 1.5167 - learning_rate: 1.0000e-04
Epoch 7/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 85s 54m

In [ ]:
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Error')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()